In [1]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from aesvdd import autoencoder
from aesvdd import model
from aesvdd import datasets

import warnings
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras import Model
from tensorflow import keras
import pandas as pd
import numpy as np

import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(os.getcwd())
warnings.filterwarnings("ignore")


Instructions for updating:
non-resource variables are not supported in the long term
/home/kael99/Documents/Code/Autoencoder_SVDD


In [8]:
# X, y = datasets.get_paysim()
# X.tofile("./data/X.dat")
# y.tofile("./data/y.dat")
X = np.fromfile("./data/X.dat").reshape(-1,9)
y = np.fromfile("./data/y.dat", dtype=np.int64)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=50000, test_size=10000)
# X_train.tofile("./data/X_train.dat")
# y_train.tofile("./data/y_train.dat")
# X_test.tofile("./data/X_test.dat")
# y_test.tofile("./data/y_test.dat")


In [9]:
tf.reset_default_graph()
ae_model = autoencoder.VAE(z_dim=5)
# ae_model = autoencoder.Vanilla_AE(H=5)

In [10]:
# pretrain
ae_model.fit(X_train, shuffle=True, epochs=1, batch_size=100, validation_data=(X_test, None), verbose=1)


Train on 50000 samples, validate on 10000 samples
50000/50000 [==============================] - 2s 30us/sample - loss: 0.8870 - val_loss: -0.8959


In [11]:
inputs = ae_model.input
outputs = ae_model.get_layer(name="code_layer").output
encoder = Model(inputs=inputs, outputs=outputs)

In [12]:
ae_svdd = model.AESVDD(encoder, input_shape=(9,), representation_dim=5, objective="soft", nu=0.05, lr=5e-4)

In [13]:
#train AESVDD
t0 = time.time()
hisory = ae_svdd.fit(X_train, X_test, y_test, epochs=2, verbose=False)
print('Train time:', time.time() - t0)

#test AESVDD
t0 = time.time()
score = ae_svdd.predict(X_test)
auc = roc_auc_score(y_test, -score)
print('Test time:', time.time() - t0)
print('------------- AUROC ----------- : %.4f' % auc)


Train time: 1.185619831085205
Test time: 0.05665254592895508
------------- AUROC ----------- : 0.6350


In [14]:
ae_svdd.save_model("3")

In [17]:
ae_svdd.sess.run(ae_svdd.R)

0.47401488

In [18]:
l = ae_svdd.keras_model.predict(X_test)
R = ae_svdd.sess.run(ae_svdd.R)
c = ae_svdd.c_np
res = np.sum(np.square(l-c), axis=1) - R**2
auc = roc_auc_score(y_test, -res)
print(auc)

0.6349579663730984
